In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Hugging face login with token


In [ ]:
from huggingface_hub import login
login()

Unzip

In [ ]:

import zipfile
import os

def unzip_folder(zip_file_path, extract_path):
  """Unzips a zip file to a specified location.

  Args:
    zip_file_path: The path to the zip file.
    extract_path: The directory to extract the zip file contents to.
  """
  try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)
    print(f"Successfully unzipped '{zip_file_path}' to '{extract_path}'")
  except FileNotFoundError:
    print(f"Error: Zip file '{zip_file_path}' not found.")
  except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")


zip_file = "/content/drive/MyDrive/archive.zip"
extract_dir = "/content/drive/MyDrive/Spider"

os.makedirs(extract_dir, exist_ok=True)

unzip_folder(zip_file, extract_dir)


In [ ]:
import pandas as pd

In [ ]:

df1 = pd.read_json("/content/drive/MyDrive/Spider/spider/train_spider.json")



In [ ]:

df2 = pd.read_json("/content/drive/MyDrive/Spider/spider/train_others.json")



In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
len(df)

In [ ]:
df.head(10)

In [ ]:
df.drop(df.columns[[2,3,5,6]], axis=1, inplace=True)

In [ ]:
df.head(10)

In [ ]:

from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.8, random_state=42)


print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Testing set size:", len(test_df))


In [ ]:
import os

base_directory = '/content/drive/MyDrive/Spider/spider/database'
def get_schema_for_db(db_id):

    schema_file_path = os.path.join(base_directory, db_id, 'schema.sql')
    if os.path.exists(schema_file_path):
        with open(schema_file_path, 'r') as file:
            for line in file:
                if "CREATE" in line.upper():
                    create_lines.append(line)
        return create_lines
    else:
        return None

df['schema'] = df['db_id'].apply(get_schema_for_db)


In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
df = df.reset_index(drop=True)


In [ ]:
from datasets import DatasetDict
fin_data=DatasetDict({
    "train":train_dataset,
    "test":test_dataset,
    "val":val_dataset
},)

In [ ]:
train_dataset[:100]

In [ ]:
from google.colab import drive
import zipfile
import os
import pandas as pd
from datasets import Dataset
from huggingface_hub import HfApi

api = HfApi(token="repo_token")

api.create_repo(repo_id="rakshithjoseph/spider_data", repo_type="dataset", exist_ok=True)
fin_data.push_to_hub("rakshithjoseph/spider_data")

